In [49]:
# get infos for an option from yahoo finance: 

import utils.yahoo as yf
import yfinance as y
from datetime import datetime
from optionlab import run_strategy, plot_pl

In [50]:
stock = "AAPL"
ls = "long"
cp = "call"

exp_dates = yf.get_options_expirations_dates(stock)
exp_date = exp_dates["Expiration Dates"][5]
exp_date = exp_date.strftime("%Y-%m-%d")

In [51]:
options = yf.get_options_list(stock, ls=ls, cp=cp, expiration_dates=[exp_date])
option = options.iloc[35]
option 

currency                                       USD
ticker                                        AAPL
contractSymbol       AAPL241115C00225000_call_long
lastTradeDate            2024-10-08 19:59:44+00:00
strike                                       225.0
lastPrice                                      8.8
bid                                            0.0
ask                                            0.0
change                                         0.0
percentChange                                  0.0
volume                                      1378.0
openInterest                                 28441
impliedVolatility                          0.00001
inTheMoney                                    True
contractSize                               REGULAR
expirationDate                          2024-11-15
tickerPrice                                225.339
ls                                            long
cp                                            call
type                           

In [52]:
# setting up the option input format : 
period_domain = "1y"

number_of_contracts = [1, 1]  # Adjust as needed
shares_per_contract = 100  # Standard for U.S. options

input_data = {
    "stock_price": option.tickerPrice,
    "start_date": datetime.now().strftime("%Y-%m-%d"),
    "target_date": option.expirationDate,
    "volatility": yf.calculate_volatility(option.ticker, period=period_domain),
    "interest_rate": yf.get_risk_free_rate(period=period_domain),
    "min_stock": yf.get_stock_min_max(option.ticker,period=period_domain)[0],
    "max_stock": yf.get_stock_min_max(option.ticker,period=period_domain)[1],
    "strategy": [
        {
            "type": option.cp,
            "strike": option.strike,
            "premium": option.lastPrice,
            "n": shares_per_contract,
            "action": "buy" if option.ls == "long" else "sell"
        },
        {
            "type": option.cp,
            "strike": option.strike,
            "premium": option.lastPrice,
            "n": shares_per_contract,
            "action": "buy" if option.ls == "long" else "sell"
        }
    ],
}

[*********************100%***********************]  1 of 1 completed


In [53]:
out = run_strategy(input_data)

In [58]:
probability_of_profit = out.probability_of_profit  # 0.32235439683910716
profit_ranges = out.profit_ranges # [(233.81, inf)]
strategy_cost = out.strategy_cost # -1760.0000000000002
minimum_return_in_the_domain = out.minimum_return_in_the_domain # -1760.0000000000002
maximum_return_in_the_domain = out.maximum_return_in_the_domain # 573.9999999999974

# per_leg_cost = out.per_leg_cost # [-880.0000000000001, -880.0000000000001] => same a strategy_cost
# implied_volatility = out.implied_volatility # [0.275, 0.275]
# in_the_money_probability = out.in_the_money_probability # [0.5271583045940538, 0.5271583045940538]
# delta = out.delta # [0.5559462710629198, 0.5559462710629198]
# gamma = out.gamma # [0.024109185969151875, 0.024109185969151875]
# theta = out.theta # [-0.14336559446059394, -0.14336559446059394]
# vega = out.vega # [0.2864600008495908, 0.2864600008495908]
# probability_of_profit_target = out.probability_of_profit_target # None
# profit_target_ranges = out.profit_target_ranges # None
# probability_of_loss_limit = out.probability_of_loss_limit # None
# average_profit_from_mc = out.average_profit_from_mc # None
# average_loss_from_mc = out.average_loss_from_mc # None
# probability_of_profit_from_mc = out.probability_of_profit_from_mc # None

#   Variable	                Type	        Aggregation
#   Per Leg Cost	            Cost	        Sum
#   Implied Volatility	        Percentage	    Mean
#   In-the-Money Probability	Probability	    Mean
#   Delta	                    Sensitivity	    Sum
#   Gamma	                    Sensitivity	    Sum
#   Theta	                    Sensitivity	    Sum
#   Vega	                    Sensitivity	    Sum


# Calculating means
mean_implied_volatility = sum(out.implied_volatility) / len(out.implied_volatility)
mean_in_the_money_probability = sum(out.in_the_money_probability) / len(out.in_the_money_probability)

total_delta = 0.0
total_gamma = 0.0
total_theta = 0.0
total_vega = 0.0
# Calculating summaries
for i in range(len(number_of_contracts)): 
    total_delta += out.delta[i] * number_of_contracts[i] * shares_per_contract
    total_gamma += out.gamma[i] * number_of_contracts[i] * shares_per_contract
    total_theta += out.theta[i] * number_of_contracts[i] * shares_per_contract
    total_vega += out.vega[i] * number_of_contracts[i] * shares_per_contract


# Summary output
summary = {
    "profit_robability": out.probability_of_profit,
    "profit_ranges" : out.profit_ranges, 
    "strategy_cost" : out.strategy_cost,
    "minimum_return_in_the_domain" : out.minimum_return_in_the_domain,
    "maximum_return_in_the_domain" : out.maximum_return_in_the_domain,
    "Cost": total_per_leg_cost,
    "Delta": total_delta,
    "Gamma": total_gamma,
    "Theta": total_theta,
    "Vega": total_vega,
}
summary

{'profit_robability': 0.31269672248268743,
 'profit_ranges': [(233.81, inf)],
 'strategy_cost': -1760.0000000000002,
 'minimum_return_in_the_domain': -1760.0000000000002,
 'maximum_return_in_the_domain': 573.9999999999974,
 'Cost': 0.0,
 'Delta': 109.11404227953795,
 'Gamma': 4.856517407015377,
 'Theta': -28.588606141582456,
 'Vega': 57.37426175576379}